In [1]:
import tweepy
import random
import time
#from interactive_conditional_samples import interact_model
import gpt_2_simple as gpt2
from textblob import TextBlob

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



ModuleNotFoundError: No module named 'textblob'

In [ ]:
def pick_topic():
    topic = random.choice(['evolution', 'gun control', 'existence of god', 'healthcare'
                          , 'climate change', 'communism'])
    return topic

def search(topic, api):
    results = api.search(q=topic, lang='en', result_type='recent', count=1, tweet_mode='extended')
#     for res in results:
#         if 
    return results
    
def generate_responses(text,sess, gpt2):
    print('Generating responses')
    responses = gpt2.generate(sess,
                              length=280,
                              temperature=0.8,
                              prefix=text+' ||| ',
                              nsamples=5,
                              batch_size=1,
                              return_as_list=True)
    print(responses)
    return responses

def select_response(responses):
    parsed = []
    for response in responses:
        try:
            response = response.split(' ||| ')[1]
        except:
            continue
        response = response[:240]
        response = response.split('<endoftext>')[0]
        before, sep, after = response.rpartition('.')
        response = before + sep
        parsed += [response]

    parsed = [x for x in parsed if 'emoticon' not in x and 'forum' not in x and x != '']
    sent_parsed = [[x,TextBlob(x).sentiment.polarity] for x in parsed]
    parsed.sort(key = lambda x: x[1], reverse = True)

    return parsed[0]

def reply(tweet, api,sess, gpt2):
    user = tweet._json['user']['screen_name']
    tweet_id = tweet._json['id']
    text = tweet._json['full_text']
    responses = generate_responses(text,sess, gpt2)
    reply_text = select_response(responses)
    
    api.update_status('@' + user + ' ' + reply_text, in_reply_to_status_id=tweet_id)
    return text, reply_text

In [3]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

Loading checkpoint checkpoint/run1/model-5047
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-5047


In [4]:
def main():
    auth = tweepy.OAuthHandler('5W4GMKnmBPmz73ofTbMSrBX5O', 'v3liOsP8k73qnqasUUbJAD0VaXyHJN6zy6tr2xdjdmuIgrjj0D')
    auth.set_access_token('977955567846281216-8YsIaPvhBtk3vQSZ5thkEvbiTTm39ro', 'h7JcHEceUN5PX8NuKYFA9f665DLMUSKpNizT1ZqFod2ei')

    api = tweepy.API(auth)
#     sess = gpt2.start_tf_sess()
#     gpt2.load_gpt2(sess, run_name='run1')
    while True:
        try:
            topic = pick_topic()
            tweet = search(topic, api)[0]
            text, reply_text = reply(tweet, api,sess, gpt2)
            print(topic)
            print('tweet: ', text)
            print('##########################')
            print('reply: ', reply_text)
            time.sleep(int(random.random()*120+300))
        except KeyboardInterrupt:
            break
        except:
            continue
        

In [ ]:
main()